# Extract particle VDFs from FLEKS output and select desired particles

This note book describes how to read FLEKS particle data output. First, you would need to install SWMF and enable FLEKS in it. 
During the time this jupyter notebook was created, the FLEKS provided some Python wrapper files on yt package to read the data output, in this case, you need to put FLEKS python helper library into the PYTHONPATH.
Later on, this wrapper files have been moved to a python pacakge called flekspy, detailed instructions can be found here: https://github.com/henry2004y/flekspy
The usages are not changed, the only difference in terms of what this file covers is the name of the pacakge changed from fleks to flekspy. flekspy may support more functionalities.

The following cell covers the importation of the pacakges. two util functions are also defined here.
1. ```rotation_matrix_from_vectors``` calculates the unit rotation matrix that rotates vec1 to vec2. This is function is needed because we need to rotate the velocity vector that aligns with the local magnetic field coordinates.
2. ```calc_vb_vectors``` constructs a local cartesian coordinate defined by the local bulk velocity and local magnetic field.

In [ ]:
import fleks, yt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from matplotlib import ticker

from matplotlib import  rcParams

rcParams['text.usetex'] = False
rcParams['font.family'] = 'serif'
rcParams['axes.linewidth'] = 2
rcParams['xtick.major.width'] = 1
rcParams['xtick.major.size'] = 4
rcParams['xtick.minor.width'] = 1
rcParams['xtick.minor.size'] = 2
rcParams['ytick.major.width'] = 1
rcParams['ytick.major.size'] = 4
rcParams['ytick.minor.width'] = 1
rcParams['ytick.minor.size'] = 2

def rotation_matrix_from_vectors(vec1, vec2):
    """
    Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """

    a = (vec1 / np.linalg.norm(vec1)).reshape(3)
    b = (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

def calc_vb_vectors(bvec, vvec):
    unit_b = (bvec / np.linalg.norm(bvec)).reshape(3)
    unit_v = (vvec / np.linalg.norm(vvec)).reshape(3)
    unit_vb = np.cross(unit_v, unit_b)
    unit_perp = np.cross(unit_b, unit_vb)
    unit_vb = unit_vb / np.linalg.norm(unit_vb)
    unit_perp = unit_perp / np.linalg.norm(unit_perp)
    return unit_b, unit_vb, unit_perp

The following cell is defining paths for the data files. ```smaple_points_dict``` stores the location, magnetic field, and local bulk velocity of a sampling point.
Notice that if we only need VDF, magnetic field and local bulk velocity are not needed. The reason we need these two data is we want to subtract the bulk velocity and project the velocity on B and VxB coordinates (this is also what MMS observation does).

In [ ]:
import pickle
import json

file_root = './'
data_file_dict = {
      '1321': {'e':'{:s}/3d_particle_region0_3_t00025100_n00102600_amrex'.format(file_root),  
               'i':'{:s}/na'.format(file_root)},
      '1336': {'e':'{:s}/3d_particle_region0_3_t00030600_n00111600_amrex'.format(file_root),
               'i':'{:s}/na'.format(file_root)},
}

sample_points_dict = {

      "A1336": { # bz max
            "center":[-9.03117547529323872,-6.84825371941971106,0.66096325359352126],
            "bvec":[0,-9.48951247597609182, 5.32121735196562717], 
            "vvec":[125.220516405888446,93.2773115522788743,109.492633538466436]},

      "B1336": { # bz max
            "center":[-9.76964755139845131,-7.32371192407870986,1.24371880616626251],
            "bvec":[0,-4.05134576555627035, 22.0761419531317387], 
            "vvec":[199.169579060765443,72.9757710693565684,37.52071508065481]},
      
      "C1336": { # bz max
            "center":[-11.4312097226351757,-7.94020753612441954,1.21509465825193352],
            "bvec":[0,1.25788633747233325,5.29431906060982449], 
            "vvec":[428.146766109107034,159.899385314072674,19.4124550787820915]},
      
      "D1336": { # bz max
            "center":[-17.2159076521259351,-8.79224491860180279,1.55253240743197751],
            "bvec":[0,0.702954102301058814,6.52342068511395912], 
            "vvec":[592.927637537339933,209.063232654131639,-24.5355648076267698]},
      
      "E1336": { # bz max
            "center":[-21.3390434103800004,-7.48264787712713009,2.12330857508734328],
            "bvec":[0,-0.114815775454145189,2.22874239718768008], 
            "vvec":[549.030277551509926,245.802692208670237,19.6825811397322603]},
      
      "F1336": { # bz max
            "center":[-23.2467629403184333,-5.94394102064707397,2.10841056981084085],
            "bvec":[0,0.438817533054258357,3.28264870758951766], 
            "vvec":[427.238250040752348,131.772730249905493,5.84478385225656893]},

}

The next cell is the code to load the dataset.
```ds = fleks.load(data_file)``` ds is is a yt dataset object.

In [ ]:
ptype = 'e'
point = "B1321"

def _unit_one(field, data):        
    res = np.zeros(data[('particles', 'p_w')].shape)
    res[:] = 1
    return res

data_file = data_file_dict[point[-4:]][ptype]
print(data_file)
ds = fleks.load(data_file)
# Add a user defined field. See yt document for more information about derived field.
ds.add_field(ds.pvar("unit_one"), function=_unit_one, sampling_type='particle')

The following cell is the core code to plot a VDF from a certain region. 

To be able to directly compare with MMS observations, particle velocities are usually projected to directions parallel/perpendicular to the local magnetic field with the local bulk velocity substracted.

Here notice that the AMRex data output is manipulated using yt package. Hence we need to create an array that stores the bulk velocity:

```python 
u_bulk = ds.arr(sample_points_dict[point]["vvec"], "code_velocity")
```

and the substraction is done by defining an internal function first and calling ```add_field```:

```python
def _vel_ux_vframe(field, data):
    res = data['particles', 'p_ux'] - u_bulk[0]
    return res
```

Similar process is applied on velocity projection.

The VDF is essentially a phase space density plot, this is done by 

```python
profile = yt.create_profile(data_source=source_region, bin_fields=[x_field,y_field], 
                                fields=z_field, n_bins=[64, 64], weight_field=None, logs=logs)
plot = yt.PhasePlot.from_profile(profile)
```
here ``` plot ``` variable is used to access the plot data, such as x, y, z for customization.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=int(max(len(sample_points_dict), 2) / 2), figsize=(14, 10))
fig.tight_layout()
point_list = reversed(sample_points_dict.keys())

for iplot, point in enumerate(point_list):
    plot = None
    
    # calculate velocity in bulk velocity frame
    u_bulk = ds.arr(sample_points_dict[point]["vvec"], "code_velocity")

    def _vel_ux_vframe(field, data):
        res = data['particles', 'p_ux'] - u_bulk[0]
        return res
    def _vel_uy_vframe(field, data):
        res = data['particles', 'p_uy'] - u_bulk[1]
        return res
    def _vel_uz_vframe(field, data):
        res = data['particles', 'p_uz'] - u_bulk[2]
        return res
    ds.add_field(ds.pvar("ux_bulk_frame"), units="code_velocity", function=_vel_ux_vframe, sampling_type='particle', force_override=True)
    ds.add_field(ds.pvar("uy_bulk_frame"), units="code_velocity", function=_vel_uy_vframe, sampling_type='particle', force_override=True)
    ds.add_field(ds.pvar("uz_bulk_frame"), units="code_velocity", function=_vel_uz_vframe, sampling_type='particle', force_override=True)

    # project velocity to B, VxB, and the third direction
    unit_b, unit_vb, unit_bvb = calc_vb_vectors(sample_points_dict[point]["bvec"], 
                                        sample_points_dict[point]["vvec"])
    def _vel_b(field, data):
        res = unit_b[0]*data[('particles', 'ux_bulk_frame')] + unit_b[1]*data[('particles', 'uy_bulk_frame')] + unit_b[2]*data[('particles', 'uz_bulk_frame')]        
        return res
    def _vel_vb(field, data):
        res = unit_vb[0]*data[('particles', 'ux_bulk_frame')] + unit_vb[1]*data[('particles', 'uy_bulk_frame')] + unit_vb[2]*data[('particles', 'uz_bulk_frame')]        
        return res
    def _vel_bvb(field, data):
        res = unit_bvb[0]*data[('particles', 'ux_bulk_frame')] + unit_bvb[1]*data[('particles', 'uy_bulk_frame')] + unit_bvb[2]*data[('particles', 'uz_bulk_frame')]        
        return res
    ds.add_field(ds.pvar("vel_b"), units="code_velocity", function=_vel_b, sampling_type='particle', force_override=True)
    ds.add_field(ds.pvar("vel_vb"), units="code_velocity", function=_vel_vb, sampling_type='particle', force_override=True)
    ds.add_field(ds.pvar("vel_bvb"), units="code_velocity", function=_vel_bvb, sampling_type='particle', force_override=True)

    center = np.array(sample_points_dict[point]["center"])
    box_size = 0.8
    left_edge = center - 0.5 * box_size
    right_edge = center + 0.5 * box_size

    var_x_name = "vel_vb"
    var_y_name = "vel_b"
    source_region = ds.box(left_edge, right_edge)

    x_field = ("particles", var_x_name)
    y_field = ("particles", var_y_name)
    z_field = ('particles','unit_one')

    logs = {x_field: False, y_field: False}
    profile = yt.create_profile(data_source=source_region, bin_fields=[x_field,y_field], 
                                fields=z_field, n_bins=[64, 64], weight_field=None, logs=logs)
    plot = yt.PhasePlot.from_profile(profile)
    plot.set_figure_size(6)
    plot.set_cmap(plot.fields[0], "jet")
    plot.set_colorbar_label(plot.fields[0], " ")

    for var_name in plot.profile.field_data: 
        val = np.array(plot.profile.field_data[var_name])
    
    x = np.array(plot.profile.x)
    y = np.array(plot.profile.y)
    xx, yy = np.meshgrid(x, y)

    cur_ax = axes.flatten()[iplot]
    cur_ax.contourf(
                        xx, yy, np.log(np.array(np.transpose(val))),
                        cmap='jet',
                        levels=60,
                        #vmin = 44.9, vmax = 55.1
                    )
    cur_ax.set_aspect('equal')
    cur_ax.set_xlim([-15000,15000])
    cur_ax.set_ylim([-15000,15000])
    cur_ax.set_xlabel(r"$V_{V \times B}\ [km/s]$")
    if iplot == 0:
        cur_ax.set_ylabel(r"$V_B\ [km/s]$")
    cur_ax.set_title(point[0])
    cur_ax.grid()

OK, now we have the VDF of particles in a specific region, what if we want to get the detailed data of the particles inside this region? There is a convenient way to directly access particle data, here is an example of accessing the ```x``` coordinate of particles in a region:
```python
np.array(yt_region['particles', 'particle_position_x'])
```
notice that, we need to firstly define a yt region. The following function ``` extract_particle_in_region``` can return a pandas dataframe that contains the particle data inside this region.

DO NOT try to call this function for entire PIC domain, it will never finish and use up your system memory.

In [ ]:
def extract_particle_in_region(yt_region):
    df_particle = pd.DataFrame.from_dict({
        'cpu': np.array(yt_region['particles', 'particle_cpu'], dtype=np.intc),
        'supid': np.array(yt_region['particles', 'particle_supid'], dtype=np.longlong),
        'id': np.array(yt_region['particles', 'particle_id'], dtype=np.longlong),
        'x': np.array(yt_region['particles', 'particle_position_x']),
        'y': np.array(yt_region['particles', 'particle_position_y']),
        'z': np.array(yt_region['particles', 'particle_position_z']),
        'ux': np.array(yt_region['particles', 'particle_velocity_x']),
        'uy': np.array(yt_region['particles', 'particle_velocity_y']),
        'uz': np.array(yt_region['particles', 'particle_velocity_z']),
        'weight': np.array(yt_region['particles', 'particle_weight']),
    })
    df_particle['energy'] = (
                                0.5 *
                                #df_particle['weight'] * 
                                (df_particle['ux']**2 + df_particle['uy']**2 + 
                                 df_particle['uz']**2) *
                                1.66054e-27 * # amu to kg
                                1e3 ** 2 # km/s to m/s squared
                                * 6.242e+18 # joule to eV
                            )
    return df_particle

point = "B1336"
center = np.array(sample_points_dict[point]["center"])
box_size = 0.8
left_edge = center - 0.5 * box_size
right_edge = center + 0.5 * box_size
yt_region = ds.box(left_edge, right_edge)
df_particle = extract_particle_in_region(yt_region)

Once you have the particle dataframe, you can select particles inside this region based on your need. Notice that to trace particle in previous and subsequent SWMF restart trees, you need to save cpu, supid and id.

In [ ]:
df_particle

In [ ]:
def project_velocity(df, b_vec, u_bulk):
  unit_b, unit_vb, unit_bvb = calc_vb_vectors(b_vec, u_bulk)
  assert(np.abs(np.linalg.norm(unit_b) - 1.0) < 1e-6)
  assert(np.abs(np.linalg.norm(unit_vb) - 1.0) < 1e-6)
  assert(np.abs(np.linalg.norm(unit_bvb) - 1.0) < 1e-6)
  df["ux_bulk_frame"] = df["ux"] - u_bulk[0]
  df["uy_bulk_frame"] = df["uy"] - u_bulk[1]
  df["uz_bulk_frame"] = df["uz"] - u_bulk[2]
  df["vel_b"] = unit_b[0]*df["ux_bulk_frame"] + unit_b[1]*df["uy_bulk_frame"] + unit_b[2]*df["uz_bulk_frame"]
  df["vel_vb"] = unit_vb[0]*df["ux_bulk_frame"] + unit_vb[1]*df["uy_bulk_frame"] + unit_vb[2]*df["uz_bulk_frame"]
  df["vel_bvb"] = unit_bvb[0]*df["ux_bulk_frame"] + unit_bvb[1]*df["uy_bulk_frame"] + unit_bvb[2]*df["uz_bulk_frame"]

  return df


In [ ]:
b_vec = sample_points_dict[point]["bvec"]
u_bulk = sample_points_dict[point]["vvec"]
df_new = project_velocity(df_particle, b_vec, u_bulk)
df_new['vel_amp'] = np.sqrt(df_new['vel_vb']*df_new['vel_vb'] + df_new['vel_b']*df_new['vel_b'])

# check low energy particles
a, b = 1500, 2000
df_new['inside_ellipse'] = (df_new['vel_vb'] / a) ** 2 + (df_new['vel_b'] / b) **2 

r = 5000
df_new['inside_circle'] = (df_new['vel_vb'] / r) ** 2 + (df_new['vel_b'] / r) **2 

In [ ]:
df_selected = df_new.loc[(df_new['inside_ellipse'] <= 1)]

In [ ]:
df_selected.to_csv('output.csv', columns=['cpu', 'supid', 'id'],index=False, 
                    header=None, sep=" ")
with open('output.csv', 'r') as original:
    data = original.read()
with open('select_particle_low_new.in', 'w') as modified:
    modified.write(str(len(df_selected))+"\n" + data)

df_selected.to_csv('df_1321_low_new.csv',index=False, sep=" ")

In [ ]:
time_list = [1334, 1335, 1337]
df_dict = {}

for time in time_list:
  df= pd.read_csv("select_particle_{:d}.out".format(time), sep="\s+")
  for key in ['x', 'y', 'z']:
    df[key] = df[key] / 6378000
  df_dict[time] = df

df_dict[1336] = df_selected

In [ ]:
fig, axes = plt.subplots(nrows=len(df_dict), ncols=1, figsize=(6, 10))

for iax, time in enumerate(sorted(df_dict.keys())):
  df = df_dict[time]
  axes[iax].scatter(df['x'], df['y'], s=0.5)
  axes[iax].set_xlim(-40, 0)
  axes[iax].set_ylim(-10, 10)
  axes[iax].annotate('{:d}'.format(time), xy =(-35, 6), fontsize=12)



In [ ]:
fig, axes = plt.subplots(nrows=len(df_dict), ncols=1, figsize=(6, 10))

for iax, time in enumerate(sorted(df_dict.keys())):
  df = df_dict[time]
  axes[iax].scatter(df['x'], df['z'], s=0.5)
  axes[iax].set_xlim(-40, 0)
  axes[iax].set_ylim(-10, 10)
  axes[iax].annotate('{:d}'.format(time), xy =(-35, 6), fontsize=12)